In [1]:
import json
import pandas as pd

In [2]:
# function to get shortened wiki articles

N_SENTENCES=3
def get_sentences(x):
    n = N_SENTENCES + 1
    sentences = x.split('. ')
    try:
        return '. '.join(sentences[:n])
    except:
        return '. '.join(sentences)
def count_sentences(x):
    try:
        return len(x.split('. '))
    except:
        return -1

In [3]:
# read pickles

wiki_en = pd.read_pickle('data/wiki_en_summaries.pkl')
wiki_fr = pd.read_pickle('data/wiki_fr_summaries.pkl')
bnf_table = pd.read_pickle('data/bnf_table_summaries.pkl')
bnf_tags = pd.read_pickle('data/bnf_tags.pkl')

named_subjects = pd.read_pickle('data/named_subjects.pkl')

# this line will change 
age_gender = pd.read_json('age_gender_labeles.json',orient='records')

In [4]:
# get name-id df (exploded)

named_subjects = named_subjects[['id','title','names']].explode('names')
named_subjects = named_subjects.set_index('id')
named_subjects = named_subjects.rename(columns = {'names':'name'})

# get name-id df (nested in list)

id_img = named_subjects.drop('title', axis=1)
id_img = id_img.reset_index()
id_img = pd.DataFrame(id_img.groupby('name')['id'].apply(list))
id_img = id_img.rename(columns={'id':'id_list'})

In [5]:
# organize dataframes, shorten texts, rename columns

wiki_en['wiki_en_text'] = wiki_en.summary.apply(get_sentences)
wiki_fr['wiki_fr_text'] = wiki_fr.summary_fr.apply(get_sentences)

wiki_en = wiki_en[['name','wiki_en_text','url','weight']]
wiki_fr = wiki_fr[['name','wiki_fr_text','url_fr','weight_fr']]
bnf_summaries = bnf_table[['name','bnf_link','gender','note']]


wiki_en = wiki_en.rename(columns={'url':'wiki_en_link', 'weight':'weight_en'})
wiki_fr = wiki_fr.rename(columns={'url':'wiki_fr_link'})

In [6]:
# link name and id defined in age_gender df

named_age_gender = named_subjects.merge(age_gender[['age','gender','id']], on='id').\
rename(columns={'age':'age_estimate', 'gender':'gender_estimate'})


# if age_gender file is re-writtened, 
#id_name = pd.read_pickle('data/id_name.pkl').rename(columns={'id':'id_list'})
#named_age_gender = id_name.merge(age_gender[['age','gender','id']], on='name').rename(columns={'age':'age_estimate', 'gender':'gender_estimate'})


In [20]:
# merge everything

data_final = pd.merge(bnf_summaries,wiki_en,how='outer')
data_final = pd.merge(data_final,wiki_fr,how='outer')
data_final = pd.merge(data_final, named_age_gender, how='outer')
data_final = data_final.merge(id_img, on='name',how='outer')
data_final = data_final.merge(bnf_tags, on='name',how='outer')


In [22]:
# redefine rank <<<<<< THIS WILL CHANGE 


data_final.weight_en.fillna(-1, inplace=True)
data_final.weight_fr.fillna(-1, inplace=True)

data_final['weight'] = data_final[['weight_en','weight_fr']].mean(axis=1)

In [23]:
data_final = data_final.sort_values(by='weight', ascending=False)

In [24]:
data_final

,name,bnf_link,gender,note,wiki_en_text,wiki_en_link,weight_en,wiki_fr_text,url_fr,weight_fr,id,title,age_estimate,gender_estimate,id_list,tags,weight
235,"Bouguereau, William (1825-1905)",http://data.bnf.fr/ark:/12148/cb12758427r#about,masculin,"Peintre, dessinateur, aquarelliste. - Décorate...",William-Adolphe Bouguereau (French pronunciati...,https://en.wikipedia.org/wiki/William-Adolphe_...,50.0,L'œuvre peint de William Bouguereau (La Rochel...,https://fr.wikipedia.org/wiki/%C5%92uvre_peint...,424.0,https://gallica.bnf.fr/ark:/12148/btv1b53050879q,Bouguereau,43.553167,0.093347,[https://gallica.bnf.fr/ark:/12148/btv1b530508...,"[décorateur, peintre, dessinateur, academiedes...",237.0
521,"Delacroix, Eugène (1798-1863)",http://data.bnf.fr/ark:/12148/cb118991616#about,masculin,"Peintre, aquarelliste, graveur et lithographe",Ferdinand Victor Eugène Delacroix ( DEL-ə-krwa...,https://en.wikipedia.org/wiki/Eug%C3%A8ne_Dela...,56.0,Eugène Delacroix est un peintre français né le...,https://fr.wikipedia.org/wiki/Eug%C3%A8ne_Dela...,83.0,https://gallica.bnf.fr/ark:/12148/btv1b53104547j,Eug Delacroix,45.458278,0.057673,[https://gallica.bnf.fr/ark:/12148/btv1b531045...,"[peintre, graveur, aquarelliste, lithographe, ...",69.5
1573,"Robespierre, Maximilien de (1758-1794)",http://data.bnf.fr/ark:/12148/cb11922216p#about,masculin,Avocat. - Homme politique. - Révolutionnaire,Maximilien François Marie Isidore de Robespier...,https://en.wikipedia.org/wiki/Maximilien_Robes...,43.0,"Maximilien de Robespierre, ou Maximilien Robes...",https://fr.wikipedia.org/wiki/Maximilien_de_Ro...,95.0,https://gallica.bnf.fr/ark:/12148/btv1b53187673w,M Robespierre Reproduction de tableau,38.010203,0.479522,[https://gallica.bnf.fr/ark:/12148/btv1b531876...,"[hommepolitique, avocat, France]",69.0
580,"Dieterle, Amélie (1870-1941)",http://data.bnf.fr/ark:/12148/cb14676815w#about,féminin,Actrice de théâtre et de cinéma. - Cantatrice,NaN,NaN,-1.0,"Amélie Diéterle, nom de scène d'Amélie Laurent...",https://fr.wikipedia.org/wiki/Am%C3%A9lie_Di%C...,129.0,https://gallica.bnf.fr/ark:/12148/btv1b53111808r,Mlle Dieterle Variétés,31.000485,0.719496,[https://gallica.bnf.fr/ark:/12148/btv1b531118...,"[actrice, cantatrice, théâtre, cinéma, France]",64.0
451,"Courbet, Gustave (1819-1877)",http://data.bnf.fr/ark:/12148/cb120519068#about,masculin,"Peintre, sculpteur et graveur. - A été aussi l...","Jean Désiré Gustave Courbet (UK: KOOR-bay, US...",https://en.wikipedia.org/wiki/Gustave_Courbet,61.0,"Gustave Courbet, né le 10 juin 1819 à Ornans e...",https://fr.wikipedia.org/wiki/Gustave_Courbet,64.0,https://gallica.bnf.fr/ark:/12148/btv1b53050548w,"Courbet, peintre",40.302552,0.051735,[https://gallica.bnf.fr/ark:/12148/btv1b530505...,"[président, sculpteur, peintre, artiste, grave...",62.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
929,Isabelle II (1830-1904 ; reine d'Espagne),http://data.bnf.fr/ark:/12148/cb120667707#about,féminin,"Reine d'Espagne de 1833 à 1868, abdique en fav...",NaN,NaN,-1.0,NaN,NaN,-1.0,https://gallica.bnf.fr/ark:/12148/btv1b531009378,Isabelle d'Espagne,35.538605,0.066641,[https://gallica.bnf.fr/ark:/12148/btv1b531009...,[Espagne],-1.0
928,"Irisson d'Hérisson, Maurice d' (1839-1893)",http://data.bnf.fr/ark:/12148/cb12525359d#about,masculin,Publiciste. - Officier,NaN,NaN,-1.0,NaN,NaN,-1.0,https://gallica.bnf.fr/ark:/12148/btv1b53131394d,Comte d'Hérisson,39.937100,0.056591,[https://gallica.bnf.fr/ark:/12148/btv1b531313...,"[officier, publiciste, France]",-1.0
927,"Invernizzi, Peppa (1840 ?-1900 ?)",http://data.bnf.fr/ark:/12148/cb15497248n#about,féminin,"Étoile, puis mime à l'Opéra de Paris. - Ne pas...",NaN,NaN,-1.0,NaN,NaN,-1.0,https://gallica.bnf.fr/ark:/12148/btv1b53162098b,Invernizzi Opéra,27.113161,0.185324,[https://gallica.bnf.fr/ark:/12148/btv1b531620...,[Italie],-1.0
926,"Invernizzi, Lotta (18..-19.. ; danseuse)",http://data.bnf.fr/ark:/12148/cb156069553#about,féminin,Danseuse à l'opéra de Paris à partir de 1880. ...,NaN,

In [25]:
data_final = data_final[['id','name','gender','id_list', 'weight','note', 'bnf_link', 'wiki_en_text', 'wiki_en_link',
        'wiki_fr_text', 'url_fr', 'age_estimate','gender_estimate','tags']]

In [ ]:
data_final.to_pickle('data/merged_dataframe.pkl')